In [1]:
import os
from itertools import combinations
from scipy.special import comb
from IPython.display import display, HTML

from tools.compare_functions import load_solutions, get_pareto_front, get_styled_table, classify_solutions

import pandas as pd
from tqdm import tqdm 
import numpy as np
import pandas as pd



# Datos

In [2]:
archive = "WorkSpace 1000_50_10"
k=10
m=50

# Definición de la búsqueda exhaustiva

In [3]:
class BusquedaExhaustivaOptimizada:
    def __init__(self, archive, k, m):
        """
        Inicializa el algoritmo de búsqueda exhaustiva optimizado.
        
        Args:
            archive (str): Nombre base del archivo de distancias (sin .csv).
            k (int): Número de puntos de suministro a seleccionar.
            m (int): Número total de puntos de suministro disponibles.
        """
        # Carga la matriz de distancias una sola vez
        folder_distances = "./data/distances/demand/"
        route_distances = os.path.join(folder_distances, f"{archive}.csv")
        df_distances_demand = pd.read_csv(route_distances, index_col=0)

        # --- MEJORA 1: Conversión a NumPy ---
        # Usamos un array de NumPy para todos los cálculos numéricos. Es mucho más rápido.
        self.dist_matrix = df_distances_demand.to_numpy()
        
        # Guardamos los nombres de las columnas para el resultado final
        self.supply_point_names = df_distances_demand.columns.to_numpy()
        
        # Parámetros del problema
        self.k = k
        self.m = m
        self.route_solutions = f"{archive}.csv"
        
        if self.m != self.dist_matrix.shape[1]:
            raise ValueError(
                f"El valor de m ({self.m}) no coincide con el número de columnas "
                f"({self.dist_matrix.shape[1]}) en el archivo de distancias."
            )
        
        # --- MEJORA 2: Estructuras de datos eficientes para el frente de Pareto ---
        # Usamos una lista para las combinaciones y un array de NumPy para sus valores.
        self.pareto_solutions = []
        self.pareto_values = np.empty((0, 3))

    def _evaluate_solution(self, supply_indices):
        """
        Calcula los 3 objetivos para una solución de forma unificada y eficiente.
        """
        # Submatriz de distancias para los puntos de suministro seleccionados
        sub_matrix = self.dist_matrix[:, supply_indices]

        # F1: Maximizar la mínima distancia cubierta (vectorizado)
        # Lo negamos para tratar todos los objetivos como minimización
        f1_value = -np.max(np.min(sub_matrix, axis=1))

        # Asignación de puntos de demanda al suministro más cercano
        assignment_indices = np.argmin(sub_matrix, axis=1)

        # F2 y F3: Conteo de asignaciones usando bincount (muy rápido)
        counts = np.bincount(assignment_indices, minlength=self.k)
        
        f2_value = np.max(counts)
        f3_value = f2_value - np.min(counts)

        return np.array([f1_value, f2_value, f3_value])

    def _update_pareto_front(self, solution, new_values):
        """
        Actualiza el frente de Pareto usando operaciones vectorizadas de NumPy.
        Asume que todos los objetivos en `new_values` son de minimización.
        """
        # 1. ¿La nueva solución es dominada por alguna del frente actual?
        #    Si alguna solución existente es mejor o igual en todos los objetivos, la nueva es dominada.
        if np.any(np.all(self.pareto_values <= new_values, axis=1)):
            return  # La nueva solución es dominada, no hacer nada

        # 2. ¿Qué soluciones existentes son dominadas por la nueva?
        #    Identificar las soluciones existentes que son peores o iguales en todos los objetivos.
        dominated_by_new_mask = np.all(new_values <= self.pareto_values, axis=1)

        # 3. Filtrar para mantener solo las soluciones no dominadas
        self.pareto_values = self.pareto_values[~dominated_by_new_mask]
        
        # Actualizar la lista de soluciones de forma eficiente
        self.pareto_solutions = [
            s for s, is_dominated in zip(self.pareto_solutions, dominated_by_new_mask) if not is_dominated
        ]

        # 4. Añadir la nueva solución no dominada al frente
        self.pareto_values = np.vstack([self.pareto_values, new_values])
        self.pareto_solutions.append(solution)
        
    def run(self):
        """
        Ejecuta la búsqueda exhaustiva optimizada.
        """
        print("Iniciando búsqueda exhaustiva OPTIMIZADA... 🚀")
        
        potential_supplies = list(range(self.m))
        num_combinations = comb(self.m, self.k, exact=True)
        print(f"Se evaluarán {num_combinations} combinaciones. ¡Esto puede tardar!")

        all_combinations = combinations(potential_supplies, self.k)
        
        pbar = tqdm(all_combinations, total=num_combinations, desc="Procesando Combinaciones", unit=" comb")
        
        for current_solution in pbar:
            solution_list = list(current_solution)
            
            # Calcular los tres objetivos de una vez
            objective_values = self._evaluate_solution(solution_list)
            
            # Actualizar el frente de Pareto
            self._update_pareto_front(solution_list, objective_values)
            
            pbar.set_postfix(pareto_count=len(self.pareto_solutions))

        # --- MEJORA 3: Post-procesamiento y guardado único ---
        print(f"\nBúsqueda completada. Se encontraron {len(self.pareto_solutions)} soluciones no dominadas. ✅")
        
        # Revertir el signo de f1 para mostrar el valor original (maximización)
        final_values = self.pareto_values.copy()
        final_values[:, 0] *= -1 

        # Crear el DataFrame final de una sola vez
        df_final = pd.DataFrame({
            'solution': [str(sorted(s)) for s in self.pareto_solutions],
            'f1': final_values[:, 0],
            'f2': final_values[:, 1],
            'f3': final_values[:, 2]
        })

        # Guardar en CSV UNA SOLA VEZ al final del proceso
        df_final.to_csv(self.route_solutions, index=False)
        print(f"Los resultados se han guardado en '{self.route_solutions}'.")
        print("\nDataFrame final de soluciones de Pareto:")
        print(df_final)

# Ejecución

In [ ]:
if __name__ == '__main__':
    # Asegúrate de que la carpeta de datos exista
    if not os.path.exists("./data/distances/demand/"):
        print("Error: La carpeta de datos no existe. Asegúrate de tener la estructura de directorios correcta.")
    else:
        # Aquí debes poner el nombre de tu archivo (sin .csv), k y m correctos
        # ¡CUIDADO! Valores grandes de k y m harán que el programa tarde muchísimo.
        analisis_exhaustivo = BusquedaExhaustivaOptimizada(archive, k, m)
        analisis_exhaustivo.run()

# Pasar el csv a txt para comparativa de soluciones

In [3]:
def guardar_pareto_txt(ruta_csv, ruta_txt):
  """
  Lee un archivo CSV con soluciones de Pareto y guarda los datos de las
  columnas 'f1', 'f2' y 'f30' en un archivo de texto con un formato específico.

  Args:
    ruta_csv (str): La ruta al archivo CSV de entrada.
    ruta_txt (str): La ruta al archivo de texto de salida.
  """
  # Lee el archivo CSV en un DataFrame de pandas
  df = pd.read_csv(ruta_csv)

  # Abre el archivo de texto en modo de escritura
  with open(ruta_txt, 'w') as f:
    # Itera sobre cada fila del DataFrame
    for _, fila in df.iterrows():
      # Extrae y redondea los valores de las columnas
      f1 = fila['f1']
      f2 = fila['f2']
      f3 = fila['f3'] # Se ajusta el valor de f30

      # Escribe los valores formateados en el archivo de texto
      f.write(f"{f1}\t{f2}\t{f3}\n")

# Ejemplo de uso de la función
guardar_pareto_txt('./Pareto_front/WorkSpace 1000_50_10.csv', './Pareto_front/WorkSpace 1000_50_10.txt')

print("¡El archivo 'pareto_formateado.txt' ha sido creado con éxito!")

¡El archivo 'pareto_formateado.txt' ha sido creado con éxito!


# Soluciones exhaustivas vs frente de pareto paper previo

In [6]:
# Carga
ruta_solution="./Pareto_front/WorkSpace 1000_50_5.csv"
ruta_pareto="./Pareto_front_paper/"+archive+".txt"
df_csv_full, df_csv_obj, df_txt = load_solutions(ruta_solution, ruta_pareto)

In [7]:
# Unir y encontrar frente de Pareto
all_solutions = np.vstack([df_csv_obj.values, df_txt.values])
pareto_mask = get_pareto_front(all_solutions)
pareto_solutions = all_solutions[pareto_mask]

# Arrays para detección de comunes
csv_array = df_csv_obj.values
txt_array = df_txt.values

# Clasificar
colors_csv = classify_solutions(df_csv_obj, pareto_solutions, txt_array)
colors_txt = classify_solutions(df_txt, pareto_solutions, csv_array)

# Mostrar
html_csv = get_styled_table(df_csv_full, colors_csv)
html_txt = get_styled_table(df_txt, colors_txt)

# Mostrar lado a lado con HTML
display(HTML(f"""
<div style="display: flex; gap: 30px;">
  <div style="flex: 1;">
    <h3>Soluciones CSV (con ID)</h3>
    {html_csv}
  </div>
  <div style="flex: 1;">
    <h3>Soluciones TXT</h3>
    {html_txt}
  </div>
</div>
"""))


,solution,f1,f2,f3
0,"[1, 4, 14, 27, 29]",379.969736,259,160
1,"[2, 8, 11, 27, 29]",481.133038,203,6
2,"[2, 17, 19, 23, 29]",458.802790,209,22
3,"[3, 7, 12, 27, 29]",442.764046,229,104
4,"[3, 9, 12, 27, 29]",442.764046,233,103
5,"[3, 9, 13, 27, 29]",433.416659,243,113
6,"[3, 10, 15, 27, 30]",441.834811,241,125
7,"[3, 12, 15, 27, 30]",450.699456,227,83
8,"[4, 14, 27, 29, 46]",377.005305,281,210
9,"[4, 14, 27, 29, 47]",377.005305,284,202


## Truncando los decimales para comparar correctamente

In [8]:
df_csv_full['f1'] = df_csv_full['f1'].astype(int)
df_csv_obj['f1'] = df_csv_obj['f1'].astype(int)

In [9]:
# Unir y encontrar frente de Pareto
all_solutions = np.vstack([df_csv_obj.values, df_txt.values])
pareto_mask = get_pareto_front(all_solutions)
pareto_solutions = all_solutions[pareto_mask]

# Arrays para detección de comunes
csv_array = df_csv_obj.values
txt_array = df_txt.values

# Clasificar
colors_csv = classify_solutions(df_csv_obj, pareto_solutions, txt_array)
colors_txt = classify_solutions(df_txt, pareto_solutions, csv_array)

# Mostrar
html_csv = get_styled_table(df_csv_full, colors_csv)
html_txt = get_styled_table(df_txt, colors_txt)

# Mostrar lado a lado con HTML
display(HTML(f"""
<div style="display: flex; gap: 30px;">
  <div style="flex: 1;">
    <h3>Soluciones CSV (con ID)</h3>
    {html_csv}
  </div>
  <div style="flex: 1;">
    <h3>Soluciones TXT</h3>
    {html_txt}
  </div>
</div>
"""))


,solution,f1,f2,f3
0,"[1, 4, 14, 27, 29]",379,259,160
1,"[2, 8, 11, 27, 29]",481,203,6
2,"[2, 17, 19, 23, 29]",458,209,22
3,"[3, 7, 12, 27, 29]",442,229,104
4,"[3, 9, 12, 27, 29]",442,233,103
5,"[3, 9, 13, 27, 29]",433,243,113
6,"[3, 10, 15, 27, 30]",441,241,125
7,"[3, 12, 15, 27, 30]",450,227,83
8,"[4, 14, 27, 29, 46]",377,281,210
9,"[4, 14, 27, 29, 47]",377,284,202


# Soluciones extraidas vs soluciones exactas

In [10]:
# Carga
ruta_solution="Solutions/Multiprocessing/"+archive+"/"+archive+".csv"
ruta_pareto="./Pareto_front/WorkSpace 1000_50_5.txt"
df_csv_full, df_csv_obj, df_txt = load_solutions(ruta_solution, ruta_pareto)

In [11]:
# Unir y encontrar frente de Pareto
all_solutions = np.vstack([df_csv_obj.values, df_txt.values])
pareto_mask = get_pareto_front(all_solutions)
pareto_solutions = all_solutions[pareto_mask]

# Arrays para detección de comunes
csv_array = df_csv_obj.values
txt_array = df_txt.values

# Clasificar
colors_csv = classify_solutions(df_csv_obj, pareto_solutions, txt_array)
colors_txt = classify_solutions(df_txt, pareto_solutions, csv_array)

# Mostrar
html_csv = get_styled_table(df_csv_full, colors_csv)
html_txt = get_styled_table(df_txt, colors_txt)

# Mostrar lado a lado con HTML
display(HTML(f"""
<div style="display: flex; gap: 30px;">
  <div style="flex: 1;">
    <h3>Soluciones CSV (con ID)</h3>
    {html_csv}
  </div>
  <div style="flex: 1;">
    <h3>Soluciones TXT</h3>
    {html_txt}
  </div>
</div>
"""))


,solution,f1,f2,f3
0,"[13, 23, 29, 45, 49]",412.506970,247,148
1,"[2, 8, 11, 27, 29]",481.133038,203,6
2,"[1, 4, 14, 27, 29]",379.969736,259,160
3,"[4, 14, 27, 29, 46]",377.005305,281,210
4,"[3, 12, 15, 27, 30]",450.699456,227,83
5,"[3, 9, 13, 27, 29]",433.416659,243,113
6,"[3, 10, 15, 27, 30]",441.834811,241,125
7,"[13, 20, 23, 29, 49]",415.120464,233,129
8,"[2, 17, 19, 23, 29]",458.802790,209,22
9,"[3, 9, 12, 27, 29]",442.764046,233,103
